In [2]:
# Cài đặt lại đúng phiên bản PyTorch và CUDA phù hợp trên Colab
!pip uninstall -y torch torchvision torchaudio
!pip uninstall -y nvidia-cublas-cu12 nvidia-cudnn-cu12 nvidia-cuda-runtime-cu12 nvidia-cuda-cupti-cu12 nvidia-cuda-nvrtc-cu12 nvidia-cufft-cu12 nvidia-curand-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 nvidia-nvjitlink-cu12

# Cài đúng bản torch + CUDA 12.4
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu124

# Cài đúng các thư viện NVIDIA CUDA phụ trợ yêu cầu bởi torch 2.6.0+cu124
!pip install \
  nvidia-cublas-cu12==12.4.5.8 \
  nvidia-cudnn-cu12==9.1.0.70 \
  nvidia-cuda-runtime-cu12==12.4.127 \
  nvidia-cuda-cupti-cu12==12.4.127 \
  nvidia-cuda-nvrtc-cu12==12.4.127 \
  nvidia-cufft-cu12==11.2.1.3 \
  nvidia-curand-cu12==10.3.5.147 \
  nvidia-cusolver-cu12==11.6.1.9 \
  nvidia-cusparse-cu12==12.3.1.170 \
  nvidia-nvjitlink-cu12==12.4.127


Found existing installation: torch 2.7.0.dev20250226+cu124
Uninstalling torch-2.7.0.dev20250226+cu124:
  Successfully uninstalled torch-2.7.0.dev20250226+cu124
Found existing installation: torchvision 0.22.0.dev20250226+cu124
Uninstalling torchvision-0.22.0.dev20250226+cu124:
  Successfully uninstalled torchvision-0.22.0.dev20250226+cu124
Found existing installation: torchaudio 2.6.0.dev20250226+cu124
Uninstalling torchaudio-2.6.0.dev20250226+cu124:
  Successfully uninstalled torchaudio-2.6.0.dev20250226+cu124
Found existing installation: nvidia-cublas-cu12 12.4.5.8
Uninstalling nvidia-cublas-cu12-12.4.5.8:
  Successfully uninstalled nvidia-cublas-cu12-12.4.5.8
Found existing installation: nvidia-cudnn-cu12 9.1.0.70
Uninstalling nvidia-cudnn-cu12-9.1.0.70:
  Successfully uninstalled nvidia-cudnn-cu12-9.1.0.70
Found existing installation: nvidia-cuda-runtime-cu12 12.4.127
Uninstalling nvidia-cuda-runtime-cu12-12.4.127:
  Successfully uninstalled nvidia-cuda-runtime-cu12-12.4.127
Found e

In [3]:
pip uninstall -y fastai


In [4]:
!pip install -qq datasets

In [5]:
!pip show torch -qq

In [6]:
!gdown 1DuwYRftQjQmQcAR4FVPH-HvGuxGi4ist
!gdown 11xZZfla8CDH54-EeUUdnAAoT2ummuEJh
!gdown 1wVyhQkhAzwod2at7Ir3tRHbdCMOszzwg

Downloading...
From: https://drive.google.com/uc?id=1DuwYRftQjQmQcAR4FVPH-HvGuxGi4ist
To: /content/train_word.conll
100% 1.42M/1.42M [00:00<00:00, 43.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=11xZZfla8CDH54-EeUUdnAAoT2ummuEJh
To: /content/test_word.conll
100% 958k/958k [00:00<00:00, 78.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wVyhQkhAzwod2at7Ir3tRHbdCMOszzwg
To: /content/dev_word.conll
100% 628k/628k [00:00<00:00, 93.8MB/s]


Định nghĩa hàm read_conll() để đọc dữ liệu NER từ file .conll

Trích xuất từng câu (sentences) và nhãn thực thể tương ứng (labels)

Lưu trữ nhãn duy nhất (unique_labels) để phục vụ huấn luyện

Đọc 3 bộ dữ liệu: train, dev, test

In [7]:
# để đọc file định dạng CoNLL
def read_conll(file_path):
    sentences = []
    sentence_labels = []
    unique_labels = set()  # To collect unique labels

    with open(file_path, 'r') as file:
        current_sentence_tokens = []
        current_sentence_labels = []

        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace, including '\n'

            # If it's an empty line, sentence boundary detected
            if not line:
                if current_sentence_tokens:  # Check if there's a sentence to append
                    sentences.append(' '.join(current_sentence_tokens))
                    sentence_labels.append(' '.join(current_sentence_labels))
                current_sentence_tokens = []  # Reset for the next sentence
                current_sentence_labels = []  # Reset for the next sentence
            else:
                line_parts = line.split()  # Split line into token and label
                current_sentence_tokens.append(line_parts[0])

                if len(line_parts) >= 2:
                    current_sentence_labels.append(line_parts[1])
                    unique_labels.add(line_parts[1])  # Add label to the set of unique labels
                else:
                    current_sentence_labels.append('O')  # Default to 'O' if no label provided

    # Append the last sentence if the file doesn't end with an empty line
    if current_sentence_tokens:
        sentences.append(' '.join(current_sentence_tokens))
        sentence_labels.append(' '.join(current_sentence_labels))

    print(f"Unique labels found: {unique_labels}")
    return sentences, sentence_labels

# Load the datasets
test_sentences, test_labels = read_conll('./test_word.conll')
dev_sentences, dev_labels = read_conll('./dev_word.conll')
train_sentences, train_labels = read_conll('./train_word.conll')

# Now, test_sentences, test_labels, dev_sentences, dev_labels, train_sentences, and train_labels are arrays of strings


Unique labels found: {'I-SYMPTOM_AND_DISEASE', 'I-ORGANIZATION', 'B-JOB', 'B-DATE', 'B-NAME', 'I-JOB', 'B-TRANSPORTATION', 'B-GENDER', 'O', 'I-PATIENT_ID', 'I-AGE', 'I-NAME', 'I-DATE', 'I-TRANSPORTATION', 'B-AGE', 'I-LOCATION', 'B-SYMPTOM_AND_DISEASE', 'B-LOCATION', 'B-ORGANIZATION', 'B-PATIENT_ID'}
Unique labels found: {'I-SYMPTOM_AND_DISEASE', 'I-ORGANIZATION', 'B-JOB', 'B-DATE', 'B-NAME', 'I-JOB', 'B-TRANSPORTATION', 'B-GENDER', 'O', 'I-PATIENT_ID', 'I-NAME', 'I-DATE', 'I-TRANSPORTATION', 'B-AGE', 'I-LOCATION', 'B-SYMPTOM_AND_DISEASE', 'B-LOCATION', 'B-ORGANIZATION', 'B-PATIENT_ID'}
Unique labels found: {'I-SYMPTOM_AND_DISEASE', 'I-ORGANIZATION', 'B-JOB', 'B-DATE', 'B-NAME', 'I-JOB', 'B-TRANSPORTATION', 'B-GENDER', 'O', 'I-PATIENT_ID', 'I-AGE', 'I-NAME', 'I-DATE', 'I-TRANSPORTATION', 'B-AGE', 'I-LOCATION', 'B-SYMPTOM_AND_DISEASE', 'B-LOCATION', 'B-ORGANIZATION', 'B-PATIENT_ID'}


In [8]:
test_sentences[1]

'Bác_sĩ Trần_Thanh_Linh , từ Bệnh_viện Chợ_Rẫy chi_viện phụ_trách đơn_nguyên hồi_sức tích_cực , cho biết " bệnh_nhân 416 " vẫn đang duy_trì ECMO , thở máy , hiện xơ phổi rất nhiều .'

In [9]:
test_labels[1]

'O O O O B-ORGANIZATION I-ORGANIZATION O O O O O O O O O O B-PATIENT_ID O O O O O O O O O O B-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE O'

In [10]:
from datasets import Dataset

# Step 1: Prepare the datasets from sentences and labels
def prepare_dataset(sentences, labels):
    return {'tokens': sentences, 'labels': labels}

train_dataset = prepare_dataset(train_sentences, train_labels)
dev_dataset = prepare_dataset(dev_sentences, dev_labels)
test_dataset = prepare_dataset(test_sentences, test_labels)

# Step 2: Convert strings of tokens and labels into arrays
def process_string_to_array(dataset):
    return {
        'tokens': [sentence.split() for sentence in dataset['tokens']],
        'labels': [label_seq.split() for label_seq in dataset['labels']]
    }

# Step 3: Process the dataset for token and label lists
train_dataset = process_string_to_array(train_dataset)
dev_dataset = process_string_to_array(dev_dataset)
test_dataset = process_string_to_array(test_dataset)

# Step 4: Convert processed datasets into Hugging Face Dataset objects
train_dataset = Dataset.from_dict(train_dataset)
dev_dataset = Dataset.from_dict(dev_dataset)
test_dataset = Dataset.from_dict(test_dataset)

# Print the size of each dataset and a sample for verification
print(f"Train dataset size: {len(train_dataset)}")
print(f"Dev dataset size: {len(dev_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print("Train dataset sample:", train_dataset[0])
print("Dev dataset sample:", dev_dataset[0])
print("Test dataset sample:", test_dataset[0])

# Step 5: Define an Example class
class Example:
    def __init__(self, words, slot_labels, guid=None):
        self.words = words
        self.slot_labels = slot_labels
        self.guid = guid

# Step 6: Convert the dataset to Example objects
def convert_to_examples(dataset):
    return [
        Example(words=tokens, slot_labels=labels, guid=i)
        for i, (tokens, labels) in enumerate(zip(dataset['tokens'], dataset['labels']))
    ]

# Convert datasets into Example objects
train_examples = convert_to_examples(train_dataset)
dev_examples = convert_to_examples(dev_dataset)
test_examples = convert_to_examples(test_dataset)


Train dataset size: 5027
Dev dataset size: 2000
Test dataset size: 3000
Train dataset sample: {'tokens': ['Đồng_thời', ',', 'bệnh_viện', 'tiếp_tục', 'thực_hiện', 'các', 'biện_pháp', 'phòng_chống', 'dịch_bệnh', 'COVID', '-', '19', 'theo', 'hướng_dẫn', 'của', 'Bộ', 'Y_tế', '.'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'O']}
Dev dataset sample: {'tokens': ['Bác_sĩ', 'Nguyễn_Trung_Nguyên', ',', 'Giám_đốc', 'Trung_tâm', 'Chống', 'độc', ',', 'Bệnh_viện', 'Bạch_Mai', ',', 'cho', 'biết', 'bệnh_nhân', 'được', 'chuyển', 'đến', 'bệnh_viện', 'ngày', '7/3', ',', 'chẩn_đoán', 'ngộ_độc', 'thuốc', 'điều_trị', 'sốt_rét', 'chloroquine', '.'], 'labels': ['O', 'O', 'O', 'O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'O', 'O', 'B-SYMPTOM_AND_DISEASE', 'I-SYMPTOM_AND_DISEASE', 'O', 'O', 'O', 'O']}
Test datas

In [11]:
import logging
logger = logging.getLogger(__name__)

import copy
import json
import logging
import os

In [12]:
def convert_examples_to_features(
    examples,
    max_seq_len,
    tokenizer,
    pad_label_id=-100,
    cls_token_segment_id=0,
    pad_token_segment_id=0,
    sequence_segment_id=0,
    mask_padding_with_zero=True,
):
    # Get special tokens from the tokenizer
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    unk_token = tokenizer.unk_token
    pad_token_id = tokenizer.pad_token_id

    # List to hold the converted features
    features = []

    for example_index, example in enumerate(examples):
        # Log progress every 5000 examples
        if example_index % 400 == 0:
            logger.info(f"Processing example {example_index} of {len(examples)}")

        # Tokenize each word and align its corresponding label
        tokens = []
        label_ids = []

        for word, label in zip(example.words, example.slot_labels):
            word_tokens = tokenizer.tokenize(word)

            # If the word cannot be tokenized, use [UNK] token
            if not word_tokens:
                word_tokens = [unk_token]

            tokens.extend(word_tokens)

            # Map string label to integer ID, apply pad_label_id for subword tokens
            label_id = label_map[label]
            label_ids.extend([label_id] + [pad_label_id] * (len(word_tokens) - 1))

        # Handle sequence truncation for [CLS] and [SEP] tokens
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:max_seq_len - special_tokens_count]
            label_ids = label_ids[:max_seq_len - special_tokens_count]

        # Add [SEP] token at the end of the sentence
        tokens.append(sep_token)
        label_ids.append(pad_label_id)
        token_type_ids = [sequence_segment_id] * len(tokens)

        # Add [CLS] token at the start of the sentence
        tokens = [cls_token] + tokens
        label_ids = [pad_label_id] + label_ids
        token_type_ids = [cls_token_segment_id] + token_type_ids

        # Convert tokens to input IDs
        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # Create attention masks (1 for real tokens, 0 for padding tokens)
        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)


        # Pad sequences to the maximum sequence length
        padding_length = max_seq_len - len(input_ids)
        input_ids += [pad_token_id] * padding_length
        attention_mask += [0 if mask_padding_with_zero else 1] * padding_length
        token_type_ids += [pad_token_segment_id] * padding_length
        label_ids += [pad_label_id] * padding_length

        # Create InputFeatures object and append it to the list of features
        features.append(
            InputFeatures(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                slot_labels_ids=label_ids,
            )
        )

    return features


In [13]:
# Define the label list (ensure that it includes all labels from your dataset)
label_list = ['B-ORGANIZATION', 'B-TRANSPORTATION', 'B-JOB', 'I-PATIENT_ID', 'B-NAME', 'I-DATE', 'O', 'B-PATIENT_ID', 'I-AGE', 'I-JOB', 'B-DATE', 'I-TRANSPORTATION', 'B-SYMPTOM_AND_DISEASE', 'I-SYMPTOM_AND_DISEASE', 'B-GENDER', 'I-NAME', 'B-LOCATION', 'I-LOCATION', 'I-ORGANIZATION', 'B-AGE']

# Create a mapping from label strings to integers
label_map = {label: i for i, label in enumerate(label_list)}


In [14]:
import json

In [15]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, token_type_ids, slot_labels_ids):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.slot_labels_ids = slot_labels_ids

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

In [16]:
!pip install -qq transformers
from transformers import AutoTokenizer

# Use the 'xlm-roberta-base' tokenizer, which uses the desired special tokens
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

tokenizer.cls_token, tokenizer.sep_token, tokenizer.unk_token, tokenizer.pad_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


('<s>', '</s>', '<unk>', 1)

In [17]:
from transformers import RobertaTokenizerFast

# Initialize the tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', add_prefix_space=True)

# Set the maximum sequence length
max_seq_len = 128  # You can adjust this based on your model/input

# Convert examples to features
train_features = convert_examples_to_features(train_examples, max_seq_len, tokenizer)
dev_features = convert_examples_to_features(dev_examples, max_seq_len, tokenizer)
test_features = convert_examples_to_features(test_examples, max_seq_len, tokenizer)


In [18]:
import torch
from torch.utils.data import Dataset

# Define a Dataset class to wrap the tokenized features for training
class NERDataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        return {
            'input_ids': torch.tensor(feature.input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(feature.attention_mask, dtype=torch.long),
            'token_type_ids': torch.tensor(feature.token_type_ids, dtype=torch.long),
            'labels': torch.tensor(feature.slot_labels_ids, dtype=torch.long),
        }

# Convert tokenized features into PyTorch datasets
train_dataset = NERDataset(train_features)
dev_dataset = NERDataset(dev_features)
test_dataset = NERDataset(test_features)


In [19]:
train_dataset[0]

{'input_ids': tensor([    0,  4236, 16948,  1376,  2023,  9085,  2590,  1215,   212,  1376,
          2023,    46,   118,  2156,   741,  1376,  2023,  6382,   282,   298,
          1215,  6873,  1376,  2023,  6382,   282, 12369,  1376,  3070,  9470,
           642,  1215,    90,  1376,  2023,  8210,   438,  3553,  1376,  2023,
         15389,   438,  1215,  3592,  1376,  2023,  6382,   282,   740,  1526,
           438,  4003,  1376,  2023,  6382,   282,  1215,  3792,  1526,   642,
          7843,  3849, 14292,  2590,  1215,   611,  1376,  2023,  3602,  2590,
           385,  1376,  2023, 13859,   611,  1215,   428,  1376,  2023,  6382,
           282,   298,  6247, 43814,   111,   753,     5,   139,  1368,  8188,
          7487,  1376,  2023,  3726,  2590,  1215,   417,  1376,  3070,  4958,
           282,   740,  1376,  2023,  6248,   102,   163,  1376,  2023,    27,
           854,  1215,    90,  1376,  3070,  9470,   479,     2,     1,     1,
             1,     1,     1,     1,   

In [20]:
from transformers import RobertaForTokenClassification

# Define the number of unique labels (ensure this matches your dataset's label set)
num_labels = len(label_list)  # e.g., the number of unique labels such as O, B-ORG, etc.

# Load the RoBERTa model for token classification
model = RobertaForTokenClassification.from_pretrained('roberta-base', num_labels=num_labels)


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
!pip install seqeval -qq

In [23]:
import os
from transformers import TrainingArguments, Trainer, EvalPrediction
from seqeval.metrics import f1_score, precision_score, recall_score

# Tự động đăng nhập wandb
os.environ["WANDB_API_KEY"] = "fb80f73fcb020dd331c4509a5851a96be928f490"  # Thêm mã API của bạn ở đây

def compute_metrics(p: EvalPrediction):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    true_preds, true_labels = [], []

    for pred, label in zip(preds, labels):
        tmp_preds, tmp_labels = [], []
        for p_i, l_i in zip(pred, label):
            if l_i != -100:
                tmp_preds.append(label_list[p_i])
                tmp_labels.append(label_list[l_i])
        true_preds.append(tmp_preds)
        true_labels.append(tmp_labels)

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
    }

training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=False,  # Không dùng evaluation_strategy
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    report_to=["none"],  # Tắt WandB
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Huấn luyện và đánh giá
for epoch in range(int(training_args.num_train_epochs)):
    print(f"\n==== Epoch {epoch + 1} ====")
    trainer.train(resume_from_checkpoint=None)
    eval_metrics = trainer.evaluate()
    print(eval_metrics)


<ipython-input-23-41ff445ecb0f>:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



==== Epoch 1 ====


Step,Training Loss
10,1.449800
20,0.793800
30,0.596800
40,0.454400
50,0.468600
60,0.374100
70,0.374400
80,0.257700
90,0.274400
100,0.247100


{'eval_loss': 0.12403533607721329, 'eval_precision': 0.9022881880024737, 'eval_recall': 0.913731016126507, 'eval_f1': 0.9079735511474135, 'eval_runtime': 13.9029, 'eval_samples_per_second': 143.855, 'eval_steps_per_second': 8.991, 'epoch': 3.0}

==== Epoch 2 ====


Step,Training Loss
10,0.061100
20,0.093600
30,0.078800
40,0.062700
50,0.087400
60,0.079100
70,0.079600
80,0.062000
90,0.071400
100,0.078400


{'eval_loss': 0.12387196719646454, 'eval_precision': 0.917910447761194, 'eval_recall': 0.9243776420854861, 'eval_f1': 0.9211326936578516, 'eval_runtime': 14.0329, 'eval_samples_per_second': 142.523, 'eval_steps_per_second': 8.908, 'epoch': 3.0}

==== Epoch 3 ====


Step,Training Loss
10,0.022600
20,0.045300
30,0.035900
40,0.036900
50,0.039900
60,0.066100
70,0.037100
80,0.034600
90,0.045500
100,0.046700


{'eval_loss': 0.1338542103767395, 'eval_precision': 0.9220455254131588, 'eval_recall': 0.9259433223735714, 'eval_f1': 0.9239903132567768, 'eval_runtime': 13.9378, 'eval_samples_per_second': 143.494, 'eval_steps_per_second': 8.968, 'epoch': 3.0}


In [24]:
trainer.save_model("./roberta-ner-finetuned")
tokenizer.save_pretrained("./roberta-ner-finetuned")


('./roberta-ner-finetuned/tokenizer_config.json',
 './roberta-ner-finetuned/special_tokens_map.json',
 './roberta-ner-finetuned/vocab.json',
 './roberta-ner-finetuned/merges.txt',
 './roberta-ner-finetuned/added_tokens.json',
 './roberta-ner-finetuned/tokenizer.json')

In [26]:
# Sau khi huấn luyện xong, đánh giá mô hình trên test set

# Tạo dataset từ file test (dựa vào NERDataset bạn đã định nghĩa từ trước)
# Assuming label_map is the correct mapping, otherwise replace with the correct variable
label_to_id = label_map
test_dataset = NERDataset(test_features) # Modify this line to use test_features

# Đánh giá mô hình trên test set
test_metrics = trainer.evaluate(test_dataset)

# In kết quả
print("\n====== Đánh giá trên TEST set ======")
print(f"Loss: {test_metrics['eval_loss']:.4f}")
print(f"Precision: {test_metrics['eval_precision']:.4f}")
print(f"Recall: {test_metrics['eval_recall']:.4f}")
print(f"F1 Score: {test_metrics['eval_f1']:.4f}")


====== Đánh giá trên TEST set ======
Loss: 0.1698
Precision: 0.9153
Recall: 0.9180
F1 Score: 0.9167


In [27]:
from seqeval.metrics import classification_report

# Lấy nhãn thực và nhãn dự đoán từ tập test
predictions, labels, _ = trainer.predict(test_dataset)
preds = predictions.argmax(-1)

true_labels = []
true_preds = []

for pred, label in zip(preds, labels):
    true_label = []
    true_pred = []
    for p_i, l_i in zip(pred, label):
        if l_i != -100:
            true_label.append(label_list[l_i])
            true_pred.append(label_list[p_i])
    true_labels.append(true_label)
    true_preds.append(true_pred)

# In báo cáo chi tiết
print("\n====== Báo cáo chi tiết theo từng thực thể ======")
print(classification_report(true_labels, true_preds, digits=4))



====== Báo cáo chi tiết theo từng thực thể ======
                     precision    recall  f1-score   support

                AGE     0.9652    0.9633    0.9643       518
               DATE     0.9790    0.9904    0.9847      1459
             GENDER     0.9379    0.9704    0.9539       405
                JOB     0.5779    0.5742    0.5761       155
           LOCATION     0.9130    0.9232    0.9181      3671
               NAME     0.8943    0.8527    0.8730       258
       ORGANIZATION     0.8165    0.8246    0.8205       707
         PATIENT_ID     0.9697    0.9814    0.9755      1665
SYMPTOM_AND_DISEASE     0.8093    0.7598    0.7838       916
     TRANSPORTATION     0.9535    0.9535    0.9535       172

          micro avg     0.9153    0.9180    0.9167      9926
          macro avg     0.8816    0.8793    0.8803      9926
       weighted avg     0.9145    0.9180    0.9161      9926

